In [19]:
import os
import time
import winsound
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import keras
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input, BatchNormalization, LeakyReLU
from tensorflow.keras.models import Model
from keras.optimizers import AdamW
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, Callback
import glob

In [20]:
print(tf.config.list_physical_devices('GPU'))
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    tf.config.set_visible_devices(gpus[0], 'GPU')

[]


In [21]:
base_path = 'dataset/'
train_dir = os.path.join(base_path, 'train')

train_csv = os.path.join(base_path, 'train', '_annotations.csv')
valid_csv = os.path.join(base_path, 'valid', '_annotations.csv')
test_csv = os.path.join(base_path, 'test', '_annotations.csv')
                        
# Function to load and preprocess images
def load_and_preprocess_image(filename, label, bbox):
    try:
        img = tf.io.read_file(filename)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [360, 360])  # VGG16 input size
        img = tf.cast(img, tf.float32)  / 255.0
        #img = tf.keras.applications.vgg16.preprocess_input(img)
        return img, label, bbox
    except tf.errors.NotFoundError:
        print(f"File not found: {filename}")
        return None, None, None

# Function to load data from CSV and create a dataset
def create_dataset(csv_file,base_dir, is_training=True):
    df = pd.read_csv(csv_file)
    unique_labels = df['class'].unique()
    print(unique_labels)
    # Encode class labels
    le = LabelEncoder()
    df['class'] = le.fit_transform(df['class'])

    filenames = df['filename'].apply(lambda x: os.path.join(base_dir, x)).values
    labels = df['class'].values
    labels_one_hot = tf.keras.utils.to_categorical(labels, num_classes=len(le.classes_))

    bboxes = df[['xmin', 'ymin', 'xmax', 'ymax']].values.astype(np.float32)
    
    print(f"Base Dir: {base_dir}")
    print(f"Labels shape before one-hot encoding: {labels.shape}")
    print(f"Labels shape after one-hot encoding: {labels_one_hot.shape}")    
    print(f"Bounding boxes shape: {bboxes.shape}")
    print(f"Number of classes: {len(le.classes_)}")
    print()
    
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels_one_hot, bboxes))
    dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.filter(lambda x, y, z: x is not None)  # Filter out None values
 
    if is_training:
        dataset = dataset.shuffle(buffer_size=1000)
    
    dataset = dataset.batch(64)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    
    return dataset, le

# Create datasets one by one for train , valid , test
train_dataset, label_encoder = create_dataset(train_csv, os.path.join(base_path, 'train'))
valid_dataset, _ = create_dataset(valid_csv, os.path.join(base_path, 'valid'), is_training=False)
test_dataset, _ = create_dataset(test_csv, os.path.join(base_path, 'test'), is_training=False)

['SMG' 'ShotGun' 'HandGun' 'Rifle']
Base Dir: dataset/train
Labels shape before one-hot encoding: (3200,)
Labels shape after one-hot encoding: (3200, 4)
Bounding boxes shape: (3200, 4)
Number of classes: 4

['HandGun' 'Rifle' 'ShotGun' 'SMG']
Base Dir: dataset/valid
Labels shape before one-hot encoding: (819,)
Labels shape after one-hot encoding: (819, 4)
Bounding boxes shape: (819, 4)
Number of classes: 4

['SMG' 'ShotGun' 'HandGun' 'Rifle']
Base Dir: dataset/test
Labels shape before one-hot encoding: (467,)
Labels shape after one-hot encoding: (467, 4)
Bounding boxes shape: (467, 4)
Number of classes: 4



In [22]:
# Create a custom VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(360, 360, 3))
for layer in base_model.layers[:-4]:
    layer.trainable = False

# Input layer
inputs = Input(shape=(360, 360, 3))

# VGG16 base model
x = base_model(inputs, training=True)

# Global average pooling
x = GlobalAveragePooling2D()(x)

# Additional fully connected layers to extend the model to 25 layers

# Layer 17: Fully Connected Layer 1 with Batch Normalization
x = Dense(512)(x)
x = BatchNormalization()(x)  # Batch normalization to stabilize training
x = LeakyReLU(negative_slope=0.1)(x)  # Leaky ReLU for better performance
x = Dropout(0.5)(x)  # Higher dropout to reduce overfitting

# Layer 18: Fully Connected Layer 2
x = Dense(256)(x)
x = BatchNormalization()(x)
x = LeakyReLU(negative_slope=0.1)(x)
x = Dropout(0.5)(x)

# Layer 19: Fully Connected Layer 3
x = Dense(128)(x)
x = BatchNormalization()(x)
x = LeakyReLU(negative_slope=0.1)(x)
x = Dropout(0.4)(x)

# Layer 20: Fully Connected Layer 4
x = Dense(64)(x)
x = BatchNormalization()(x)
x = LeakyReLU(negative_slope=0.1)(x)

# Output layers
# Layer 24: Output layer for class predictions
class_output = Dense(4, activation='softmax', name='class_output')(x)

# Layer 25: Output layer for bounding box regression
bbox_output = Dense(4, name='bbox_output')(x)

# Create the final model
model = Model(inputs=inputs, outputs=[class_output, bbox_output])

# Compile the model with a learning rate scheduler
optimizer = AdamW(learning_rate=0.0001, weight_decay=0.0001)

# Compile the model with a lower learning rate for fine-tuning
model.compile(optimizer=optimizer,
              loss={'class_output': 'categorical_crossentropy', 'bbox_output': 'mse'},
              loss_weights={'class_output': 1.0, 'bbox_output': 1.0},
              metrics={'class_output': 'accuracy','bbox_output': 'mae'})

# Learning rate scheduler and early stopping
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)

# Summary of the model (to check number of layers)
model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7       │ (None, 360, 360,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ vgg16 (Functional)  │ (None, 11, 11,    │ 14,714,688 │ input_layer_7[0]… │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 512)       │          0 │ vgg16[0][0]       │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_12 (Dense)    │ (None, 512)       │    262,656 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 512)       │      2,048 │ dense_12[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_12      │ (None, 512)       │          0 │ batch_normalizat… │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_9 (Dropout) │ (None, 512)       │          0 │ leaky_re_lu_12[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_13 (Dense)    │ (None, 256)       │    131,328 │ dropout_9[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256)       │      1,024 │ dense_13[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_13      │ (None, 256)       │          0 │ batch_normalizat… │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_10          │ (None, 256)       │          0 │ leaky_re_lu_13[0… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_14 (Dense)    │ (None, 128)       │     32,896 │ dropout_10[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128)       │        512 │ dense_14[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_14      │ (None, 128)       │          0 │ batch_normalizat… │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_11          │ (None, 128)       │          0 │ leaky_re_lu_14[0… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_15 (Dense)    │ (None, 64)        │      8,256 │ dropout_11[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64)        │        256 │ dense_15[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_15      │ (None, 64)        │          0 │ batch_normalizat… │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 15,154,184 (57.81 MB)

 Trainable params: 7,517,000 (28.68 MB)

 Non-trainable params: 7,637,184 (29.13 MB)

In [23]:
for batch in train_dataset.take(1):
    print(f"Batch shapes: {[x.shape for x in batch]}")

# Check data shapes
for batch in train_dataset.take(1):
    images, labels, bboxes = batch
    print(f"Batch images shape: {images.shape}")
    print(f"Batch labels shape: {labels.shape}")
    print(f"Batch bounding boxes shape: {bboxes.shape}")

Batch shapes: [TensorShape([64, 360, 360, 3]), TensorShape([64, 4]), TensorShape([64, 4])]
Batch images shape: (64, 360, 360, 3)
Batch labels shape: (64, 4)
Batch bounding boxes shape: (64, 4)


In [24]:
class LastThreeEpochCheckpoint(Callback):
    def __init__(self, checkpoint_dir='model/', save_freq='epoch', verbose=1):
        super(LastThreeEpochCheckpoint, self).__init__()
        self.checkpoint_dir = checkpoint_dir
        self.save_freq = save_freq
        self.verbose = verbose
        
        if not os.path.exists(self.checkpoint_dir):
            os.makedirs(self.checkpoint_dir)

    def on_epoch_end(self, epoch, logs=None):
        # Save model for the current epoch
        file_path = os.path.join(self.checkpoint_dir, f'epoch_checkpoint_{epoch+1:02d}.keras')
        self.model.save_weights(file_path)
        if self.verbose:
            print(f"\nCheckpoint saved for epoch {epoch+1}: {file_path}")
        
        # Keep only the last 3 checkpoints
        checkpoints = sorted(glob.glob(os.path.join(self.checkpoint_dir, 'epoch_checkpoint_*.keras')))
        if len(checkpoints) > 3:
            os.remove(checkpoints[0])  # Remove the oldest checkpoint
            if self.verbose:
                print(f"Removed checkpoint: {checkpoints[0]}")

In [25]:
# Checkpoint to save the last three epochs only
checkpoint = LastThreeEpochCheckpoint(checkpoint_dir='model/', verbose=1)

# Check for saved checkpoints to resume training from the last checkpoint
initial_epoch = 0
checkpoints = sorted(glob.glob(os.path.join('model/', 'epoch_checkpoint_*.keras')))
if checkpoints:
    latest_checkpoint = checkpoints[-1]  # Get the latest checkpoint
    print(f"Loading weights from {latest_checkpoint}")
    model.load_weights(latest_checkpoint)
    
    # Extract epoch number from the checkpoint filename
    initial_epoch = int(latest_checkpoint.split('_')[2].split('.')[0])

# Continue training from the last saved epoch
history = model.fit(train_dataset,
                    initial_epoch=initial_epoch,
                    epochs=100,  # Total epochs
                    validation_data=valid_dataset,
                    callbacks=[lr_scheduler, early_stopping, checkpoint])


Epoch 1/100


c:\Users\dilda\Documents\major_project\wds_project\wdsenv\Lib\site-packages\keras\src\optimizers\base_optimizer.py:678: UserWarning: Gradients do not exist for variables ['kernel', 'bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


InvalidArgumentError: Graph execution error:

Detected at node gradient_tape/compile_loss/mse/mul/BroadcastGradientArgs defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "c:\Users\dilda\Documents\major_project\wds_project\wdsenv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "c:\Users\dilda\Documents\major_project\wds_project\wdsenv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "c:\Users\dilda\Documents\major_project\wds_project\wdsenv\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "c:\Users\dilda\Documents\major_project\wds_project\wdsenv\Lib\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "C:\Users\dilda\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py", line 608, in run_forever

  File "C:\Users\dilda\AppData\Local\Programs\Python\Python311\Lib\asyncio\ba

In [ ]:
model.save('model/final_model.keras')